In [1]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime

In [32]:
PATH = r'\\192.168.0.75\솔림헬프'
PATH_LOG_SUCCESS = r'\\192.168.0.75\스캔파일\스캔파일 log\success'
PATH_LOG_FAIL = r'\\192.168.0.75\스캔파일\스캔파일 log\fail'


# PATH = r'C:\Users\SL\Desktop\test'
# PATH_LOG_SUCCESS = r'C:\Users\SL\Desktop\test\log\success'
# PATH_LOG_FAIL = r'C:\Users\SL\Desktop\test\log\fail'



file_list = os.walk(PATH)
log = []
error = []
p_day211 = re.compile(r'[\s_]?\((\d\d)(\d)(\d)\)')
p_day212 = re.compile(r'[\s_]?\((\d\d)(\d)(\d\d)\)')
p_day411 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d)\)')
p_day412 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d\d)\)')

for root, __dirs__, files in tqdm(file_list) :
    try :
        for f in files :
            n = f
            if p_day211.search(f) :
                n = p_day211.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("211", f, n)
            elif p_day212.search(f) :
                n = p_day212.sub('_\g<1>0\g<2>\g<3>', n)
                #print("212", f, n)
            elif p_day411.search(f) :
                n = p_day411.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("411", f, n)
            elif p_day412.search(f) :
                n = p_day412.sub('_\g<1>0\g<2>\g<3>', n)
                #print("412", f, n)
            else :pass

            if n != f :
                log.append(ff.re_name(join(root,f), join(root,n)))


    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(log, PATH_LOG_SUCCESS)
ff.write_log_csv(error, PATH_LOG_FAIL)



0it [00:00, ?it/s]

log 파일이 비어있습니다.


In [8]:
PATH = r'\\192.168.0.75\솔림헬프\2.양도통지서'
for r, d, fs in tqdm(os.walk(PATH)) :
    for f in fs :
        if re.search("양도통지서 2_", f) :
            print(re.search("양도통지서 2_", f))

0it [00:00, ?it/s]

<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
<re.Match object; span=(13, 21), match='양도통지서 2_'>
